In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('response_times.csv')
df.head()

In [ ]:
stats = df.groupby('endpoint')['tempo_resposta'].describe()

print("Estatísticas descritivas por endpoint:")
print(stats)

In [ ]:
import seaborn as sns

plt.figure(figsize=(12, 10))
endpoints = df['endpoint'].unique()

for i, endpoint in enumerate(endpoints, 1):
    plt.subplot(2, 2, i)
    subset = df[df['endpoint'] == endpoint]
    sns.histplot(subset['tempo_resposta'], bins=30, kde=True, color='skyblue')
    plt.title(f'Distribuição - {endpoint}')
    plt.xlabel('Tempo de Resposta (s)')
    plt.ylabel('Frequência')

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

for endpoint in df['endpoint'].unique():
    subset = df[df['endpoint'] == endpoint]
    # Suaviza com média móvel (rolling)
    avg_per_iteration = subset.groupby('iteracao')['tempo_resposta'].mean().rolling(window=20).mean()
    plt.plot(avg_per_iteration, label=endpoint)

plt.title('Evolução dos Tempos de Resposta ao Longo das Iterações (Suavizado)')
plt.xlabel('Iteração')
plt.ylabel('Tempo Médio de Resposta (s)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
mean_times.plot(kind='bar', figsize=(10, 6), color='skyblue')
plt.title('Média dos Tempos de Resposta por Endpoint')
plt.xlabel('Endpoint')
plt.ylabel('Tempo Médio de Resposta (segundos)')
plt.xticks(rotation=45)
plt.show()